In [4]:
%%cython -a
# cython: boundscheck=False, wraparound=False, nonecheck=False, cdivision=True, initializedcheck=False
# distutils: language = c++
from libcpp cimport bool
from libcpp.vector cimport vector
from libcpp.algorithm cimport unique
import numpy as np
cimport numpy as np

# Function to solve the ESPPRC problem
def ESPPRC(double[:,:,:] r, double[:] r_max):
    cdef int num_nodes = r.shape[0]
    cdef int num_resources = r.shape[2]
    cdef size_t i, Nxt_p, MAX = 1000000
    cdef unsigned short[:] I, V
    cdef unsigned short[:, :] P
    cdef double[:, :] R
    cdef bool[:] O, N
    cdef bool[:, :] visit, U
    cdef double[:] lb
    cdef double ub

    # Initialize
    I, N, visit, U, P, R, O, V, lb, ub = initialize(num_nodes, num_resources)
    
    # Propagation loop
    while ContinuePropagation(O, N):
        Nxt_p = getNextParentIndx(MAX, O, R, N)
        if Nxt_p == -1:
            break  # No valid parent index found
        propagate(Nxt_p, I, N, visit, U, P, R, O, V, lb, ub, r, r_max)
        AllLabelDominance(MAX, num_nodes, num_resources, I, N, visit, U, P, R, O, V, lb)

    # Construct solution
    return [[P[i, j] for j in range(I[i] + 1)] for i in range(P.shape[0]) if not N[i]]

# Function to initialize all data structures
cdef tuple initialize(int n, int n_res):
    cdef long long MAX = 100000
    cdef np.ndarray[np.uint16_t, ndim=1] I = np.zeros(MAX, dtype=np.uint16)
    cdef np.ndarray[np.npy_bool, ndim=1, cast=True] N = np.ones(MAX, dtype=np.bool)
    cdef np.ndarray[np.npy_bool, ndim=1, cast=True] O = np.zeros(MAX, dtype=np.bool)
    cdef np.ndarray[np.npy_bool, ndim=2, cast=True] U = np.ones((MAX, n), dtype=np.bool)
    cdef np.ndarray[np.npy_bool, ndim=2, cast=True] visit = np.zeros((MAX, n), dtype=np.bool)
    cdef np.ndarray[np.double_t, ndim=2] R = np.zeros((MAX, n_res), dtype=np.double)
    cdef np.ndarray[np.uint16_t, ndim=2] P = np.zeros((MAX, n), dtype=np.uint16)
    cdef np.ndarray[np.double_t, ndim=1] lb = np.zeros(MAX, dtype=np.double)
    cdef double ub = 1000.0

    # Initialize first parent
    U[:, 0] = False
    N[0] = False
    O[0] = True
    visit[0, 0] = True

    return I, N, visit, U, P, R, O, np.zeros(MAX, dtype=np.uint16), lb, ub

# Function to find the next parent index for propagation
cdef size_t getNextParentIndx(long long MAX, bool[:] O, double[:,:] R, bool[:] N):
    cdef double min_cost = 1e9
    cdef size_t min_index = -1
    cdef size_t i

    for i in range(MAX):
        if O[i] and not N[i] and R[i, 0] < min_cost:
            min_cost = R[i, 0]
            min_index = i
    return min_index

# Function to check if propagation should continue
cdef bool ContinuePropagation(bool[:] O, bool[:] N):
    return any(O[i] and not N[i] for i in range(O.shape[0]))

# Propagation function
cdef void propagate(size_t Nxt, unsigned short[:] I, bool[:] N, bool[:,:] visit, bool[:,:] U,
                    unsigned short[:,:] P, double[:,:] R, bool[:] O, unsigned short[:] V,
                    double[:] lb, double ub, double[:,:,:] r, double[:] r_max):
    cdef int num_resources = r.shape[2]
    cdef int num_nodes = U.shape[1]
    cdef size_t j, k, ch_index
    cdef bool flag

    parent_vertex = V[Nxt]
    for j in range(num_nodes):
        if U[Nxt, j]:
            ch_index = getNextChildIndx(N)
            if ch_index == -1:
                continue
            N[ch_index] = False

            # Update resources
            R[ch_index, :num_resources] = R[Nxt, :num_resources] + r[parent_vertex, j, :num_resources]

            # Update path
            P[ch_index, :I[Nxt] + 1] = P[Nxt, :I[Nxt] + 1]
            I[ch_index] = I[Nxt] + 1
            P[ch_index, I[ch_index]] = j

            # Update visitation
            visit[ch_index, :] = visit[Nxt, :]
            visit[ch_index, j] = True

            # Update reachable nodes
            U[ch_index, :] = U[Nxt, :]
            U[ch_index, j] = False
            for k in range(num_nodes):
                if U[ch_index, k]:
                    if any(R[ch_index, l] + r[j, k, l] > r_max[l] for l in range(1, num_resources)):
                        U[ch_index, k] = False

            # Check feasibility of new path
            flag = all(R[ch_index, l] < r_max[l] / 2 for l in range(1, num_resources)) and I[ch_index] >= num_nodes / 2
            O[ch_index] = flag
            V[ch_index] = j

    O[Nxt] = False

# Function for dominance rules
cdef void AllLabelDominance(...):
    # Add your logic for label dominance here
    pass



Error compiling Cython file:
------------------------------------------------------------
...
    cdef bool flag

    parent_vertex = V[Nxt]
    for j in range(num_nodes):
        if U[Nxt, j]:
            ch_index = getNextChildIndx(N)
                       ^
------------------------------------------------------------

C:\Users\rmirjali\.ipython\cython\_cython_magic_e5c4198f5c27994269cec2a95fc2a88e351eaea6.pyx:85:23: undeclared name not builtin: getNextChildIndx

Error compiling Cython file:
------------------------------------------------------------
...
            if ch_index == -1:
                continue
            N[ch_index] = False

            # Update resources
            R[ch_index, :num_resources] = R[Nxt, :num_resources] + r[parent_vertex, j, :num_resources]
                                                                 ^
------------------------------------------------------------

C:\Users\rmirjali\.ipython\cython\_cython_magic_e5c4198f5c27994269cec2a95fc2a88e35

In [1]:
%load_ext cython


In [8]:
%%cython -a
cimport numpy as np

def compare_U(np.ndarray[np.npy_bool, ndim=1] U_1, np.ndarray[np.npy_bool, ndim=1] U_2):
    cdef size_t i
    for i in range(U_1.shape[0]):
        if U_1[i] <= U_2[i]:
            return False
    return True


Content of stdout:
_cython_magic_8a4657e3a749d69d7b0ff41dc9802cb014f2151b.c
c:\Users\rmirjali\AppData\Local\anaconda3\Lib\site-packages\numpy\core\include\numpy\npy_1_7_deprecated_api.h(14) : Warning Msg: Using deprecated NumPy API, disable it with #define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION
C:\Users\rmirjali\.ipython\cython\_cython_magic_8a4657e3a749d69d7b0ff41dc9802cb014f2151b.c(4296): warning C4018: '<': signed/unsigned mismatch
   Creating library C:\Users\rmirjali\.ipython\cython\Users\rmirjali\.ipython\cython\_cython_magic_8a4657e3a749d69d7b0ff41dc9802cb014f2151b.cp312-win_amd64.lib and object C:\Users\rmirjali\.ipython\cython\Users\rmirjali\.ipython\cython\_cython_magic_8a4657e3a749d69d7b0ff41dc9802cb014f2151b.cp312-win_amd64.exp
Generating code
Finished generating code

In [19]:
def compare_U_np(U_1, U_2):
    return np.all(U_1<=U_2)
%timeit compare_U_np(eps.U[1,:], eps.U[2,:])

3.33 μs ± 48.1 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [8]:
%%cython -a


# distutils: language = c++ 

from libc.math cimport floor
from cython.parallel import prange
from libcpp.vector cimport vector
from libcpp cimport bool

# Calculate the reduced cost
cdef double reduced_cost(double weight, vector[double]& pi, vector[double]& a, double res_max):
    cdef double total_cost = 0
    cdef size_t i
    for i in range(len(pi)):
        total_cost += pi[i] * a[i]
    return floor(total_cost / res_max) * weight

# Check if propagation should continue
cdef bool ContinuePropagation(vector[bool]& O, vector[bool]& N):
    cdef size_t i
    for i in range(len(O)):
        if O[i] and not N[i]:
            return True
    return False

# Helper function to determine dominance between two paths
cdef bool dominates(
    int i, int j, vector[vector[double]]& R, 
    vector[double]& lb, vector[vector[bool]]& visit, int num_resources
):
    cdef int k

    # Dominance conditions
    if lb[i] > lb[j]:
        return False
    for k in range(num_resources):
        if R[i][k] > R[j][k]:
            return False
    for k in range(len(visit[i])):
        if visit[i][k] and not visit[j][k]:
            return False
    return True

# Function to apply label dominance across all paths
cdef void AllLabelDominance(
    int n, int num_resources, 
    vector[int]& I, vector[bool]& N, 
    vector[vector[bool]]& visit, vector[vector[bool]]& U, 
    vector[vector[double]]& R, vector[bool]& O, 
    vector[int]& V, vector[double]& lb
):
    cdef size_t i, j
    cdef vector[int] to_remove
    to_remove.reserve(len(O))

    for i in range(len(O)):
        if not O[i]:
            continue
        for j in range(i + 1, len(O)):
            if not O[j]:
                continue
            
            # Compare paths based on resources
            if dominates(i, j, R, lb, visit, num_resources):
                to_remove.push_back(j)
            elif dominates(j, i, R, lb, visit, num_resources):
                to_remove.push_back(i)
                break

    # Remove dominated paths
    for i in reversed(to_remove):
        O[i] = False

# Function to propagate labels through the graph
cdef void propagate(
    size_t Nxt, vector[int]& I, vector[bool]& N, 
    vector[vector[bool]]& visit, vector[vector[bool]]& U, 
    vector[vector[int]]& P, vector[vector[double]]& R, 
    vector[bool]& O, vector[int]& V, 
    vector[double]& lb, double ub,
    double[:,:,:] r, double[:] r_max
):
    cdef int num_resources = r.shape[2]
    cdef int num_nodes = len(U[0])
    cdef size_t j, ch_index
    cdef int parent_vertex

    parent_vertex = V[Nxt]
    for j in range(num_nodes):
        if U[Nxt][j]:
            # Create a new child index
            ch_index = len(I)
            I.push_back(I[Nxt] + 1)
            N.push_back(False)
            visit.push_back(visit[Nxt][:])
            visit[ch_index][j] = True
            U.push_back(U[Nxt][:])
            U[ch_index][j] = False

            # Update resources
            R.push_back([R[Nxt][k] + r[parent_vertex, j, k] for k in range(num_resources)])

            # Check resource feasibility
            for k in range(num_resources):
                if R[ch_index][k] > r_max[k]:
                    continue

            # Update path
            P.push_back(P[Nxt][:])
            P[ch_index].push_back(j)

            # Check dominance and feasibility
            flag = all(R[ch_index][l] <= r_max[l] for l in range(num_resources))
            O.push_back(flag)


Content of stdout:
_cython_magic_22e8e84be18c578893564500ee6cd3a987aea653.cpp
C:\Users\rmirjali\.ipython\cython\_cython_magic_22e8e84be18c578893564500ee6cd3a987aea653.cpp(18267): warning C4018: '<': signed/unsigned mismatch
C:\Users\rmirjali\.ipython\cython\_cython_magic_22e8e84be18c578893564500ee6cd3a987aea653.cpp(18347): warning C4018: '<': signed/unsigned mismatch
C:\Users\rmirjali\.ipython\cython\_cython_magic_22e8e84be18c578893564500ee6cd3a987aea653.cpp(18410): warning C4305: '=': truncation from 'int' to 'bool'
C:\Users\rmirjali\.ipython\cython\_cython_magic_22e8e84be18c578893564500ee6cd3a987aea653.cpp(18587): warning C4305: '=': truncation from 'int' to 'bool'
C:\Users\rmirjali\.ipython\cython\_cython_magic_22e8e84be18c578893564500ee6cd3a987aea653.cpp(18652): warning C4018: '<': signed/unsigned mismatch
C:\Users\rmirjali\.ipython\cython\_cython_magic_22e8e84be18c578893564500ee6cd3a987aea653.cpp(18695): warning C4018: '<': signed/unsigned mismatch
C:\Users\rmirjali\.ipython\cytho

In [23]:
import numpy as np
import networkx as nx

n = 40
q = np.random.randint(1,5,(1,n)).repeat(n,axis=0)*0.5
q +=q.T
q *=0.5
t = np.random.randint(1,11,(n,n))
t = (t+t.T)*0.5
c = np.round(np.random.random((n,n))*10-5,2)
c = c+c.T
c *=0.5
r = np.zeros((n,n,3))
r[:,:,0] = c
r[:,:,1] = t
r[:,:,2] = q
r_max = np.array([26,8],dtype=np.double)
r = r.astype(np.double)
# ESPPRC(r, r_max)

G = nx.DiGraph()
G.add_weighted_edges_from([(i,j,r[i,j,0]) for i in range(n) for j in range(n) if i!=j])

In [ ]:
import numpy as np
from typing import  List, Tuple, Dict, Callable,Any


import numpy as np


def val_w(arr, args):
    r,r_max= args
    # Ensure arr is a boolean mask or convert it to one if necessary
    # arr = arr.astype(bool)
    U,R = arr[:r.shape[0]], arr[r.shape[0]:]
    U = U.astype(bool)
    v = r[U,:, :][:, U, :].min(axis=0)
    v,w = v[:,0].T,v[:,1:].T
    r_remains = r_max - R[1:]
    return kp(v,w,r_remains)+R[0]

def kp(arr, args):
    r,r_max= args
    # Ensure arr is a boolean mask or convert it to one if necessary
    # arr = arr.astype(bool)
    U,R,i = arr[:r.shape[0]], arr[r.shape[0]:r.shape[0]+r.shape[2]],arr[-1]
    U = U.astype(bool)
    v = r[U,:, :][:, U, :].min(axis=0)
    v,w = v[:,0].T,v[:,1:].T
    r_remains = r_max - R[1:]-r[int(i),U,1:].min(axis=0)
    
    n_items = len(v)
    n_resources = w.shape[0]
    
    # Step 1: Compute scores (value-to-mean resource usage ratio)
    scores = (v/abs(v.max()))* np.mean(w/r_remains[:, None], axis=0)  # Higher score = better "bang for buck"

    # Step 2: Sort items by score (descending order)
    sorted_indices = np.argsort(scores)
    sorted_values, sorted_resources = v[sorted_indices], w[:, sorted_indices]

    # Step 3: Cumulative resource usage check
    cumulative_usage = np.cumsum(sorted_resources, axis=1)
    fits_constraints = np.all(cumulative_usage <= r_remains[:, None], axis=0)
    if not np.all(fits_constraints):
        return np.inf
    # Step 4: Find the maximum feasible subset
    last_valid_index = np.where(fits_constraints)[0][-1]
    selected = np.zeros_like(v, dtype=bool)
    selected[sorted_indices[:last_valid_index + 1]] = True

    # Step 5: Compute results
    total_value = np.sum(v[selected])
    
    return total_value+R[0]#+r[int(i),U,0].min()

def Cost_function(r:np.ndarray,args:Any)->np.ndarray:
    # row_reduction = args['row_reduction'] 
 
    # r[:,:,0]+= (np.floor(r[:,:,3:])*(((row_reduction.reshape(1,1,row_reduction.size)).repeat(r.shape[0],axis=0)).repeat(r.shape[1],axis=1))).sum(axis=2)
    return r
def Res_function(R:np.ndarray,args:Any)->np.ndarray:
    func = args['func']
    R_:np.ndarray = np.zeros_like(R)
    for i,f in enumerate(func):
        if f is not None:
            R_[:,:,i] = f(R[:,:,i])
    return R_
def LB(U,args):
    r = args[0]
    r_ = (r[U,:,0][:,U]).min(axis=0)
    
    # row_ind,col_ind = linear_sum_assignment(r_)
    # return r[row_ind,col_ind,0].sum()
    return (r_[r_<=0]).sum()

class ESP:
    def __init__(self,
                 r_max:np.ndarray,
                 r:np.ndarray)->None:
                #  type_r:np.ndarray,
                #  cost_function:Callable,
                #  Res_function:Callable,
                #  args:Any) -> None:
        __slots__ = 'MAX','c','r','n','n_res','r_max','U','R','N','O','I','V','path','current','best_path','best_cost'
        self.MAX :int = 1000_000
        self.r:np.ndarray = r 
        self.n: int = r.shape[0]
        self.n_res:int  = r.shape[-1]
        self.r_max:np.ndarray = r_max
        self.I:np.ndarray = np.zeros(self.MAX,dtype=np.uint8)
        self.U:np.ndarray = np.ones((self.MAX,self.n),dtype=bool)
        self.R:np.ndarray = np.zeros((self.MAX,self.n_res),dtype=np.float16) #0:cost , 1: r[c,t,q]
        self.O:np.ndarray = np.ones(self.MAX,dtype=bool)# True: not reached half-point or child-up
        self.N:np.ndarray = np.ones(self.MAX,dtype=bool)# True: has not assigned a label
        self.V:np.ndarray = np.zeros(self.MAX,dtype=np.uint8) #[vertex]
        self.P:np.ndarray = np.zeros((self.MAX,n),dtype=np.uint8) #path
        self.LB: np.ndarray = np.zeros(self.MAX,dtype=np.float16)
        self.UB:float = np.inf
        self.visit:np.ndarray = np.zeros((self.MAX,self.n),dtype=np.uint8)
        self.U[:,0] = False #False: not reachable, True: reachable
        self.N[0] = False
        self.visit[0,0] = 1
        self.min_values:np.ndarray = np.min(self.r[:,:,0],axis=1)
        self.current:np.ndarray =  np.where(~self.N)[0]
        self.best_path:List[List[int]] = []
        self.best_cost:List[float] = []
        
    def generate(self)->None:
        p:np.ndarray = np.where(self.O&~self.N)[0]#[:int(1000/self.n)]
        n_child:np.ndarray =self.U[p,:].sum(axis=1).astype(int) #get of number of children
        num_child:int = int(n_child.sum())
        new_child_index:np.ndarray = self.N.nonzero()[0][:num_child]
        vertex_child:np.ndarray= np.where(self.U[p,:])[1] # the corresponding vertex
        vertex_parent:np.ndarray= np.repeat(self.V[p],n_child).astype(int)
        parent_index:np.ndarray = np.repeat(p,n_child).astype(int)
        
        self.O[p] = False #set parents closed since they all childed up
        self.N[new_child_index] = False #set new children as not new anymore
        self.V[new_child_index] = vertex_child # set the vertex of new children

        self.P[new_child_index,:] = self.P[parent_index,:] # set the path of new children
        # print(self.I[parent_index]+1)
        self.P[new_child_index,self.I[parent_index]+1] = vertex_child # set the path of new children
        self.I[new_child_index] = self.I[parent_index]+1

        self.U[new_child_index,vertex_child] = False #make children unreachable
        self.R[new_child_index,:] = self.r[vertex_parent,vertex_child,:]+self.R[parent_index,:] 
        # self.R[new_child_index,0] = self.R[new_child_index,1]#+(np.floor(self.pi.reshape(1,len(self.pi))*self.R[new_child_index,2].reshape(num_child,1))*self.xi.reshape(1,len(self.pi))).sum(axis=1)
        self.U[new_child_index,:] &=self.U[parent_index,:]
        # self.U[new_child_index,:] &=(self.Res_function(np.repeat(self.R[new_child_index,np.newaxis , 1:],repeats=self.n,axis=1)+self.r[vertex_child,:,:],self.args)<=self.r_max).all(axis=2)
        self.U[new_child_index,:] &= ((self.R[new_child_index,np.newaxis,1:]+self.r[vertex_child,:,1:])<=self.r_max).all(axis=2)
        self.O[new_child_index[np.any(self.R[new_child_index,1:]>=self.r_max/2,axis=1)]] = False
        
        self.LB[new_child_index] = np.apply_along_axis(LB_estimator, axis=1, arr=np.hstack((self.U[new_child_index, :],self.R[new_child_index,:],vertex_child.reshape(num_child,1))), args=(self.r,self.r_max) )
        self.visit[new_child_index,:] += self.visit[parent_index,:]
        self.visit[new_child_index,vertex_child] = 1
        
        # self.LB[new_child_index] = np.apply_along_axis(LB, axis=1, arr=self.U[new_child_index,:], args=(self.r,))+self.R[new_child_index,0]+self.r[vertex_child,:,0].min(axis=1)
        # # self.LB[new_child_index] = np.apply_along_axis(KP, 1, r_)+self.R[new_child_index,0]
        drop_index:np.ndarray = new_child_index[np.where(self.LB[new_child_index]>self.UB)[0]]
        self.N[drop_index] = True
        self.V[drop_index] = 0
        self.R[drop_index,:] = 0
        self.U[drop_index,:] = True
        self.O[drop_index] = True
        self.P[drop_index,:] = 0
        self.visit[drop_index,:] = 0
        self.I[drop_index] = 0
        self.LB[drop_index] = 0

        
    def sort_labels(self)->None:
        current = np.where(~self.N)[0]
        idx:np.ndarray = current[np.lexsort([self.R[current,1],(1-self.U[current,:]).sum(axis=1),
                                                  self.R[current,0],self.V[current]])]
        i:int = idx.size
        self.LB[:i] = self.LB[idx]
        self.LB[i:] = 0
        self.R[:i,:]= self.R[idx,:]
        self.R[i:,:] = 0
        self.N[:i] = False
        self.N[i:] = True
        self.V[:i]= self.V[idx]
        self.V[i:] = 0
        self.U[:i,:] = self.U[idx,:]
        self.U[i:,:] = True
        self.O[:i]= self.O[idx]
        self.O[i:] = True
        self.P[:i,:] = self.P[idx,:]
        self.P[i:,:] = 0
        self.visit[:i,:] = self.visit[idx,:]
        self.visit[i:,:] = 0
        self.I[:i] = self.I[idx]
        self.I[i:] = 0
    
    def prune_labels(self)->None:
        idx:np.ndarray = np.where(self.LB>self.UB)[0]
        self.N[idx] = True
        self.V[idx] = 0
        self.R[idx,:] = 0
        self.U[idx,:] = True
        self.O[idx] = True
        self.P[idx,:] = 0
        self.visit[idx,:] = 0
        self.I[idx] = 0
        self.LB[idx] = 0
        self.sort_labels()  
    

    def domination(self)->None:
        for i in range(1,self.n):
            idx = np.where(self.V[~self.N]==i)[0]
            r= np.hstack((self.R[idx,:],1-self.U[idx,:]))
            n_ = r.shape[0]
            ind_0, ind_1 = np.triu_indices(n_,1)
            i_ = idx[ind_1[np.where(np.all(r[ind_1,:] >= r[ind_0,:],axis=1))[0]]]
            self.N[i_] = True
            self.V[i_] = 0
            self.R[i_,:] = 0
            self.U[i_,:] = True
            self.O[i_] = True
            self.P[i_,:] = 0
            self.visit[i_,:] = 0
            self.I[i_] = 0
            self.LB[i_] = 0
        self.sort_labels()
    
        
    def solve(self)->None:
        while np.any(self.O&~self.N):
            self.generate()
            # self.prune_labels()
            self.sort_labels()
            self.domination()
            self.concat()
        self.best_path = [[int(j) for j in p] for p in self.best_path]
        self.best_cost = [float(c) for c in self.best_cost]
    
    def concat(self)->None:
        for i in range(1,self.n):
            J:np.ndarray = np.where(self.V==i)[0]
            n_ = J.size
            R:np.ndarray = self.R[J,:]
            V:np.ndarray = self.visit[J,:]
            ind_0, ind_1 = np.triu_indices(n_,1)
            r_comp:np.ndarray = R[ind_0, :] + R[ind_1, :]
            res_V:np.ndarray = V[ind_0, :] + V[ind_1, :]
            res_V[:,0]-=1
            res_V[:,i]-=1

            f = np.where(np.all(r_comp<=np.insert(self.r_max,0,0),axis=1)&np.all( res_V<=1,axis=1))[0]
            if f.size > 0:

                i_s = r_comp[f, 0].argmin()
                fs, bs = J[ind_0[f[i_s]]], J[ind_1[f[i_s]]]
                path_f, path_b = self.P[fs][:self.I[fs]+1].tolist(), self.P[bs][:self.I[bs]+1].tolist()
                path_b.reverse()
                self.best_path.append(path_f + path_b[1:])
                self.best_cost.append(r_comp[f[i_s],0])
                if r_comp[f[i_s],0] < self.UB:
                    self.UB = r_comp[f[i_s],0]  
                    print(f"New Best Cost: {self.UB}")
        

        # r = r_[num_resources+n_items+1:].reshape(n_items, n_items, num_resources+1)
        # n = U.sum()
        # values = -(r[U,:,0][:,U]).min(axis=0)
        # resource_consumption = ((r[U,:,1:][:,U,:]).min(axis=0)).reshape(num_resources,n)/resource_limits
        # # Step 1: Compute scores (value-to-mean resource usage ratio)
        # mean_resource_usage = np.mean(resource_consumption, axis=0)  # Average resource usage per item
        # scores = values / mean_resource_usage  # Higher score = better "bang for buck"

        # # Step 2: Sort items by score (descending order)
        # sorted_indices = np.argsort(-scores)  # Indices of items sorted by score
        # sorted_resources = resource_consumption[:,sorted_indices]  # Resource usage in sorted order
        # sorted_values = values[sorted_indices]  # Values in sorted order

        # # Step 3: Cumulative resource usage check
        # # Compute cumulative resource usage for all items
        # cumulative_usage = np.cumsum(sorted_resources, axis=1)  # Cumulative sum along items

        # fits_constraints = np.all(cumulative_usage <= resource_limits, axis=1)  # True if all resources fit

        # # Step 4: Find the maximum feasible subset
        # # Find the last item that fits within resource constraints
        # last_valid_index = np.where(fits_constraints)[0][-1]  # Last valid index

        # # Select the corresponding items
        # selected = np.zeros(n, dtype=bool)
        # selected[sorted_indices[:last_valid_index + 1]] = True  # Mark selected items
        

        # # Step 5: Compute results
        # total_value = np.sum(values[selected])
        # # remaining_capacity = resource_limits - np.sum(resource_consumption[selected], axis=0)
        # return -total_value

eps = ESP(r_max,r)

eps.solve()

In [25]:
LB_grb = np.zeros_like(eps.LB)
LB_grb[~eps.N]= np.apply_along_axis(LB_estimator, axis=1, arr=np.hstack((eps.U[~eps.N, :],eps.R[~eps.N,:],eps.V[~eps.N].reshape((1-eps.N).sum(),1))), args=(r,r_max) )


In [ ]:
LB_estimator(arr,(r,r_max))

4.82 ms ± 97.1 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [1]:
from cspy import BiDirectional
import networkx as nx
def LB_estimator(arr,args):
    r,r_max= args
    # Ensure arr is a boolean mask or convert it to one if necessary
    # arr = arr.astype(bool)
    U,R,vertex = arr[:r.shape[0]], arr[r.shape[0]:r.shape[0]+r.shape[2]],arr[-1]
    U = U.astype(bool)
    x = np.where(U)[0]
    G = nx.DiGraph(n_res = r_max.size)
    x = (np.append(x,vertex)).astype(int)
    for i in x:
        for j in x:
            if i!=j:
                G.add_edge("Source" if i==vertex else i,"Sink" if j==vertex else j,res_cost = r[i,j,1:], weight =r[i,j,0])
    
    alg =BiDirectional(G,min_res= [0]*r_max.size ,max_res=(r_max - R[1:]).tolist(),elementary=True)    
    alg.run()
    if alg.total_cost is None:
        return np.inf
    return R[0]+alg.total_cost

In [15]:
r

NameError: name 'r' is not defined

In [44]:
LB_grb[8081],LB_grb[8093]

(-47.53, -37.62)

In [ ]:
# %%cython -a

# # distutils: language = c++

# from libc.stdlib cimport malloc, free
# from libc.string cimport memset
# from libcpp.vector cimport vector
# from libcpp.algorithm cimport copy
# from libcpp.pair cimport pair

# from libcpp.unordered_map cimport unordered_map
# import numpy as np
# cimport numpy as np
# cimport cython

# @cython.boundscheck(False)
# @cython.wraparound(False)

# cdef void find_paths(int[:, :] graph_arcs, double[:,:] graph_data,  int num_arcs, int src, int dest,
#                      int[:] visited, int[:] path, int path_index,
#                      double[:] resources, double[:] resource_limits,
#                      int num_resources, list all_paths):
#     """
#     Recursive function to find all paths from src to dest with resource constraints.
#     """
#     cdef int i, head, tail, r
#     cdef bint valid
#     visited[src] = 1
#     path[path_index] = src
#     path_index += 1

#     if src == dest:
#         all_paths.append([path[i] for i in range(path_index)])
#     else:
#         for i in range(num_arcs):
#             head = graph_arcs[i, 0]
#             tail = graph_arcs[i, 1]
#             if head == src and not visited[tail]:
#                 valid = 1
#                 for r in range(num_resources):
#                     resources[r] += graph_data[i, r]
#                     if resources[r] > resource_limits[r]:
#                         valid = 0

#                 if valid:
#                     find_paths(graph_arcs, graph_data, num_arcs, tail, dest, visited, path, path_index, resources, resource_limits, num_resources, all_paths)

#                 # Backtrack resource consumption
#                 for r in range(num_resources):
#                     resources[r] -= graph_data[i, r]

#     # Backtrack
#     visited[src] = 0
#     path_index -= 1

# cdef void propage(int direction, int parent_index, int parent_node,
#                     vector[double]& r_max, 
#                     vector[vector[int]]& adj_list,
#                     double[:,:,:] graph_data,
#                     vector[vector[vector[int]]]& L_p,
#                     vector[vector[vector[double]]]& L_r, 
#                     vector[vector[vector[int]]]& L_reach,
#                     vector[vector[int]]& L_h,
#                     vector[vector[int]]& L_ChildUp): 
#     cdef int i, j, k, ind, child_node
#     cdef int num_resources = r_max.size()+1
    
#     cdef vector[int] child_path 
#     cdef vector[double] child_resource 
#     cdef vector[int] child_reachables 
#     cdef int child_halfpoint

#     for i in range(adj_list[parent_node].size()):
#         if L_reach[parent_node][parent_index][adj_list[parent_node][i]] == 1:

#             child_node = adj_list[parent_node][i]
#             #build a new path
#             child_path = L_p[parent_node][parent_index][:] 
#             child_path.push_back(child_node)
#             #build new resources and reachables
#             child_resource = L_r[parent_node][parent_index][:]
#             child_reachables = L_reach[parent_node][parent_index][:]
#             child_reachables[child_node] = 0
#             child_halfpoint = 0
#             for j in range(num_resources):
#                 if direction == 1:
#                     child_resource[j] += graph_data[parent_node, child_node, j]
#                 else:
#                     child_resource[j] -= graph_data[child_node, parent_node, j]

#             for  i in adj_list[child_node]:
#                 if child_reachables[i]==1:
#                     for j in range(1,num_resources):
#                         if (direction ==1 and child_resource[j]+graph_data[child_node,i,j] > r_max[j-1]) or (direction ==0 and child_resource[j]-graph_data[i,child_node,j] < 0):
#                             child_reachables[i] = 0
#                             break
#             for j in range(1,num_resources):
#                 if (direction==1 and child_resource[j]>= r_max[j-1]/2) or (direction ==0  and child_resource[j]<= r_max[j-1]/2):
#                     child_halfpoint = 1
#                     break
                            
#             #check if the child label is dominated by any of the existing labels
#             k = 0
#             while k < L_p[child_node].size():
                
#                     ind = dominanceCheck(child_resource, L_r[child_node][k], child_reachables, L_reach[child_node][k])
#                     if ind == 1:
#                         break
#                     elif ind == 2:
#                         L_p[child_node].erase(L_p[child_node].begin()+k)
#                         L_r[child_node].erase(L_r[child_node].begin()+k)
#                         L_reach[child_node].erase(L_reach[child_node].begin()+k)
#                         L_h[child_node].erase(L_h[child_node].begin()+k)
#                     else:
#                         k+=1
               

#             if ind!=1:
#                 # TODO: need to 
#                 L_ChildUp[child_node].push_back(0)
#                 L_ChildUp[parent_node][parent_index] = 1
#                 L_p[child_node].push_back(child_path)
#                 L_r[child_node].push_back(child_resource)
#                 L_reach[child_node].push_back(child_reachables)
#                 L_h[child_node].push_back(child_halfpoint)
#     L_ChildUp[parent_node][parent_index] = 1
    

        
# cdef int dominanceCheck(vector[double]& l1_res, vector[double]& l2_res, vector[int]& l1_reach, vector[int]& l2_reach):
#     cdef int i, ind
#     ind = which_label_first(l1_res, l2_res)
#     if ind == 1:
#         for i in range(l1_res.size()):
#             if l1_res[i] > l2_res[i]:
#                 return 0
#         for i in range(l1_reach.size()):
#             if l1_reach[i] > l2_reach[i]:
#                 return 0
#         return 1
#     else:
#         for i in range(l1_res.size()):
#             if l1_res[i] < l2_res[i]:
#                 return 0
#         for i in range(l1_reach.size()):
#             if l1_reach[i] < l2_reach[i]:
#                 return 0
#         return 2
    
    

# cdef int which_label_first(vector[double]& l1_res, vector[double]& l2_res):
#     cdef int i
#     for i in range(l1_res.size()):
#         if l1_res[i] < l2_res[i]:
#             return 1
#         elif l1_res[i]==l2_res[i]:
#             return which_label_first(l1_res[i+1:], l2_res[i+1:])
#         else:
#             return 2

       
# cdef int ContinuouPropagate(vector[vector[int]]& L_ChildUp, vector[vector[int]]& L_h, int num_nodes):
#     cdef int i, j
#     for i in range(num_nodes):
#         for j in range(L_ChildUp[i].size()):
#             if L_h[i][j] + L_ChildUp[i][j]<=1:
#                 return 1
#     return 0

# def generate_all_paths(int[:,:] graph_arcs, double[:, :, :] graph_data, int num_nodes, int src, int dest,
#                        double[:] resource_limits):
#     """
#     Generate all possible paths in a graph from source to destination considering resource constraints.
#     Returns:
#         List of all paths satisfying resource constraints.
#     """
#     cdef int num_arcs = graph_data.shape[0]
#     cdef int num_resources = len(resource_limits)+1
#     cdef int i, j, v
    
#     cdef int mom2calc[3]
#     mom2calc[:] = [1, 2, 3]
#     cdef vector[double] r_max
#     cdef vector[vector[vector[int]]] fw_p,bw_p # Labels'  paths
#     cdef vector[vector[vector[double]]] fw_r,bw_r # Labels' resources consumed: each vector is [c, r1, r2, ..., rn]
#     cdef vector[vector[vector[int]]] fw_reach,bw_reach # Labels'  reachables: each vector is [False, True, True, ..., True] 
#     cdef vector[vector[int]] fw_h, bw_h # Labels' halfpoints: each element is a boolean: True if label reached half-point or otherwie False
#     cdef vector[vector[int]] fw_childUp, bw_childUp # Labels' Childup Status: each element is a boolean: True if label has all child-up otherwie False

#     cdef vector[vector[int]] Out_list, In_list # Outgoing and Incoming arcs for each node

#     for i in range(num_nodes):
#         Out_list.push_back(vector[int]())
#         In_list.push_back(vector[int]())
#         fw_p.push_back(vector[vector[int]]())
#         bw_p.push_back(vector[vector[int]]())
#         fw_r.push_back(vector[vector[double]]())
#         bw_r.push_back(vector[vector[double]]())
#         fw_reach.push_back(vector[vector[int]]())
#         bw_reach.push_back(vector[vector[int]]())
#         fw_h.push_back(vector[int]())
#         bw_h.push_back(vector[int]())
#         fw_childUp.push_back(vector[int]())
#         bw_childUp.push_back(vector[int]())

#     for i in range(num_arcs):
#         Out_list[graph_arcs[i, 0]].push_back(graph_arcs[i, 1])
#         In_list[graph_arcs[i, 1]].push_back(graph_arcs[i, 0])
    
#     for i in range(num_resources-1):
#         r_max.push_back(resource_limits[i])
    
#     #Build the first forward and backward labels
#     fw_r[src].push_back(vector[double](num_resources, 0))
#     bw_r[dest].push_back(vector[double](num_resources, 0))
#     for j in range(1,num_resources+1):
#         bw_r[dest][0][j] = r_max[j-1]
#     fw_reach[src].push_back(vector[int](num_nodes, 1))
#     bw_reach[dest].push_back(vector[int](num_nodes, 1))
#     fw_reach[src][0][src] = 0
#     bw_reach[dest][0][dest] = 0
#     fw_p[src].push_back(vector[int](1, src))
#     bw_p[dest].push_back(vector[int](1, dest))
#     fw_h[src].push_back(0)
#     bw_h[dest].push_back(0)
#     fw_childUp[src].push_back(0)
#     bw_childUp[dest].push_back(0)

#     propage(direction=1, parent_index = 0, parent_node = src, 
#              r_max = r_max, adj_list = Out_list, graph_data = graph_data, L_p = fw_p, L_r = fw_r,
#              L_reach = fw_reach, L_h = fw_h, L_ChildUp = fw_childUp)
#     propage(direction = 0, parent_index = 0, parent_node = dest, r_max = r_max, adj_list = In_list,
#          graph_data = graph_data, L_p = bw_p, L_r = bw_r, L_reach = bw_reach, L_h = bw_h, L_ChildUp = bw_childUp)
    
#     # # Call the recursive function
#     # find_paths(graph_arcs,graph_data, num_arcs, src, dest, visited, path, path_index, resources, resource_limits, num_resources, all_paths)
#     for v in range(num_nodes):
#         if v!=src and v!=dest:
#             for i in range(fw_p[v].size()):
#                 if fw_childUp[v][i] == 0 and fw_h[v][i] == 0:
#                     print("here forward")
#                     propage(direction=1, parent_index = i, parent_node = v,r_max = r_max, adj_list = Out_list, graph_data = graph_data, L_p = fw_p, L_r = fw_r, L_reach = fw_reach, L_h = fw_h, L_ChildUp = fw_childUp)
#                     break

                    
#             for i in range(bw_p[v].size()):
#                 if bw_childUp[v][i] == 0 and bw_h[v][i] == 0:
#                     print("here backward")
#                     propage(direction = 0, parent_index = i, parent_node = v, r_max = r_max, adj_list = In_list,graph_data = graph_data, L_p = bw_p, L_r = bw_r, L_reach = bw_reach, L_h = bw_h, L_ChildUp = bw_childUp)
#                     break
    
#     return {i:[fw_p[i][j][k] for k in range(fw_p[i][j].size())] for i in range(fw_p.size()) for j in range(fw_p[i].size())}

In [ ]:
# cdef int num_resources = r_max.size() + 1

# cdef void build_new_path_resources_reachables(
#     int parent_node, int parent_index, int child_node, int direction,
#     double[:,:,:] graph_data, vector[vector[vector[int]]]& L_p,
#     vector[vector[vector[double]]]& L_r, vector[vector[vector[int]]]& L_reach,
#     vector[int]& child_path, vector[double]& child_resource, vector[int]& child_reachables,
#     int& child_halfpoint, int num_resources):

#     child_path = L_p[parent_node][parent_index][:]
#     child_path.push_back(child_node)

#     child_resource = L_r[parent_node][parent_index][:]
#     child_reachables = L_reach[parent_node][parent_index][:]
#     child_reachables[child_node] = 0
#     child_halfpoint = 0

#     for j in range(num_resources):
#         if direction == 1:
#             child_resource[j] += graph_data[parent_node, child_node, j]
#         else:
#             child_resource[j] -= graph_data[child_node, parent_node, j]

# cdef void update_reachables(
#     int child_node, int direction, double[:,:,:] graph_data,
#     vector[int]& child_reachables, vector[double]& child_resource, int num_resources):

#     for i in adj_list[child_node]:
#         if child_reachables[i] == 1:
#             for j in range(1, num_resources):
#                 if (direction == 1 and child_resource[j] + graph_data[child_node, i, j] > r_max[j-1]) or (direction == 0 and child_resource[j] - graph_data[i, child_node, j] < 0):
#                     child_reachables[i] = 0
#                     break

# cdef int check_halfpoint(
#     int direction, vector[double]& child_resource, int num_resources):

#     for j in range(1, num_resources):
#         if (direction == 1 and child_resource[j] >= r_max[j-1] / 2) or (direction == 0 and child_resource[j] <= r_max[j-1] / 2):
#             return 1
#     return 0

# cdef void check_dominance(
#     int child_node, vector[double]& child_resource, vector[int]& child_reachables,
#     vector[vector[vector[int]]]& L_p, vector[vector[vector[double]]]& L_r,
#     vector[vector[vector[int]]]& L_reach):

#     k = 0
#     while k < L_p[child_node].size():
#         ind = dominanceCheck(child_resource, L_r[child_node][k], child_reachables, L_reach[child_node][k])
#         if ind == 1:
#             break
#         elif ind == 2:
#             L_p[child_node].erase(L_p[child_node].begin() + k)
#             L_r[child_node].erase(L_r[child_node].begin() + k)
#             L_reach[child_node].erase(L_reach[child_node].begin() + k)
#         else:
#             k += 1

# cdef void process_node(
#     int parent_node, int parent_index, int direction, double[:,:,:] graph_data,
#     vector[vector[vector[int]]]& L_p, vector[vector[vector[double]]]& L_r,
#     vector[vector[vector[int]]]& L_reach, vector[vector[int]]& adj_list):

#     cdef int i, child_node
#     cdef vector[int] child_path 
#     cdef vector[double] child_resource 
#     cdef vector[int] child_reachables 
#     cdef int child_halfpoint

#     for i in range(adj_list[parent_node].size()):
#         if L_reach[parent_node][parent_index][adj_list[parent_node][i]] == 1:
#             child_node = adj_list[parent_node][i]
#             print(f"Processing child node: {child_node}")

#             build_new_path_resources_reachables(
#                 parent_node, parent_index, child_node, direction, graph_data,
#                 L_p, L_r, L_reach, child_path, child_resource, child_reachables,
#                 child_halfpoint, num_resources)

#             update_reachables(child_node, direction, graph_data, child_reachables, child_resource, num_resources)

#             child_halfpoint = check_halfpoint(direction, child_resource, num_resources)

#             check_dominance(child_node, child_resource, child_reachables, L_p, L_r, L_reach)

In [5]:
import numpy as np
import random

# Example graph data: [head, tail, weight, resource1, resource2]
n = 5
graph_arcs = np.array([[i,j] for i in range(n) for j in range(n) if i!=j], dtype=np.int32)

graph_data = np.random.randint(1,5, (n,n,3)).astype(np.float64)
for i in range(3):
    # graph_data[:,:,i] += graph_data[:,:,i].T
# Define constraints
num_nodes = 11
src = 0
dest = 10
resource_limits = np.array([15.0, 13.0], dtype=np.float64)  # Constraints on resources 1 and 2

# Generate all paths
paths = generate_all_paths(graph_arcs, graph_data, num_nodes, src, dest, resource_limits)
print("Valid paths:", paths)


here forward
here forward
Valid paths: {0: [0], 1: [0, 1], 2: [0, 2], 3: [0, 3], 4: [0, 4]}


: 

In [58]:
def find_paths_py(graph_data, num_nodes, src, dest, visited, path, path_index,
                  resources, resource_limits, num_resources, all_paths):
    """
    Recursive function to find all paths from src to dest with resource constraints in Python.
    """
    visited[src] = True
    path[path_index] = src
    path_index += 1

    if src == dest:
        all_paths.append(path[:path_index])
    else:
        for row in graph_data:
            head, tail = int(row[0]), int(row[1])
            if head == src and not visited[tail]:
                valid = True
                for r in range(num_resources):
                    resources[r] += row[2+ r]
                    if resources[r] > resource_limits[r]:
                        valid = False

                if valid:
                    find_paths_py(graph_data, num_nodes, tail, dest, visited, path, 
                                  path_index, resources, resource_limits, num_resources, all_paths)

                # Backtrack resource consumption
                for r in range(num_resources):
                    resources[r] -= row[2+ r]

    # Backtrack
    visited[src] = False
    path_index -= 1


def generate_all_paths_py(graph_data, num_nodes, src, dest, resource_limits):
    """
    Generate all possible paths in a graph from source to destination considering resource constraints.

    Arguments:
        graph_data: 2D list where each row represents [head, tail, weight, resource_1, resource_2, ...].
        num_nodes: Total number of nodes in the graph.
        src: Source node index.
        dest: Destination node index.
        resource_limits: List of resource constraints.

    Returns:
        List of all paths satisfying resource constraints.
    """
    visited = [False] * num_nodes
    path = [0] * num_nodes
    resources = [0.0] * len(resource_limits)
    all_paths = []
    path_index = 0
    num_resources = len(resource_limits)

    # Start recursion
    find_paths_py(graph_data, num_nodes, src, dest, visited, path, path_index,
                  resources, resource_limits, num_resources, all_paths)

    return all_paths


In [59]:
paths = generate_all_paths_py(graph_data_, num_nodes, src, dest, resource_limits)
print("Valid paths:", paths)

Valid paths: [[0, 1, 2, 3, 9], [0, 1, 2, 9], [0, 1, 3, 2, 9], [0, 1, 3, 4, 9], [0, 1, 3, 5, 9], [0, 1, 3, 6, 9], [0, 1, 3, 7, 6, 9], [0, 1, 3, 7, 9], [0, 1, 3, 8, 9], [0, 1, 3, 9], [0, 1, 4, 3, 9], [0, 1, 4, 5, 9], [0, 1, 4, 8, 9], [0, 1, 4, 9], [0, 1, 5, 2, 9], [0, 1, 5, 3, 9], [0, 1, 5, 4, 3, 9], [0, 1, 5, 4, 9], [0, 1, 5, 6, 9], [0, 1, 5, 7, 3, 9], [0, 1, 5, 7, 6, 9], [0, 1, 5, 7, 9], [0, 1, 5, 8, 3, 9], [0, 1, 5, 8, 9], [0, 1, 5, 9], [0, 1, 6, 9], [0, 1, 7, 6, 9], [0, 1, 8, 2, 9], [0, 1, 8, 3, 9], [0, 1, 8, 5, 9], [0, 1, 8, 9], [0, 1, 9], [0, 2, 1, 3, 9], [0, 2, 1, 5, 9], [0, 2, 1, 9], [0, 2, 3, 1, 9], [0, 2, 3, 4, 9], [0, 2, 3, 5, 9], [0, 2, 3, 7, 9], [0, 2, 3, 9], [0, 2, 5, 9], [0, 2, 6, 1, 9], [0, 2, 6, 9], [0, 2, 7, 1, 9], [0, 2, 7, 3, 9], [0, 2, 7, 5, 9], [0, 2, 7, 6, 1, 9], [0, 2, 7, 6, 4, 9], [0, 2, 7, 6, 9], [0, 2, 7, 9], [0, 2, 8, 9], [0, 2, 9], [0, 3, 1, 5, 9], [0, 3, 1, 8, 9], [0, 3, 1, 9], [0, 3, 2, 1, 5, 9], [0, 3, 2, 1, 9], [0, 3, 2, 6, 1, 9], [0, 3, 2, 6, 9], [0, 3, 

In [60]:
%timeit generate_all_paths(graph_arcs, graph_data, num_nodes, src, dest, resource_limits)


989 μs ± 85.1 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [61]:

%timeit generate_all_paths_py(graph_data_, num_nodes, src, dest, resource_limits)

104 ms ± 1.14 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [16]:
%%cython -a
# distutils: language = c++


from libc.stdlib cimport malloc, free
from libc.string cimport memset
from libcpp.vector cimport vector
from libcpp.algorithm cimport copy
from libcpp.pair cimport pair
from libcpp.unordered_map cimport unordered_map
import numpy as np
cimport numpy as np

def run(int num_resources, int num_nodes, int num_arcs, double[:] r_max,
             double[:,:,:] r, double[:,:] c, int[:, :] arcs):
    cdef int fw_p[1000000][100][100] 
    cdef int bw_p[1000000][100][100]

    cdef double fw_r[1000000][100][100]
    cdef double bw_r[1000000][100][100]

    cdef double fw_c[1000000][100]
    cdef double bw_c[1000000][100]

    cdef bint fw_h[1000000][100]
    cdef bint bw_h[1000000][100]
    
    cdef bint fw_reach[1000000][100][100]
    cdef bint bw_reach[1000000][100][100]
    
    cdef bint fw_childup[1000000][100] #0: not childup, 1: childup
    cdef bint bw_childup[1000000][100]

    cdef bint fw_labeled[1000000][100] #0: not assigned a label , 1: assinged a label
    cdef bint bw_labeled[1000000][100]

    cdef size_t fw_idx[1000000][100]
    cdef size_t bw_idx[1000000][100]

    cdef size_t fw_idx_max[100], bw_idx_max[100] # what is the maximum index assigned to a label so far

    cdef vector[vector[int]] OutList, InList
    cdef size_t i, j, k

    #TODO add edges to OutList and InList
    #TODO write propagate

# cdef void Propagate(bint direction, size_t parent_index, int p_v,
#                      double[:,:,:]& r, double[:]& r_max, vector[vector[int]] adj,
#                      double[:,:,:]& L_r, int[:,:,:]& L_p, bint[:,:,:]& L_h, bint[:,:,:]& L_reach, bint[:,:,:]& L_open):
#     cdef int i,j,k, v
#     for i in range(adj[p_v].size()):
#         v = adj[p_v][i]
#         for j in range(1000000):
           
 
cdef void add_node(bint drct, size_t prnt_i, int prnt_v, size_t chd_i, size_t chd_v,
                    int num_resources, int num_nodes,
                    double[:,:]& c, double[:,:,:]& r, double[:]& r_max, vector[vector[int]]& adj,
                     double[:,:,:]& L_r, int[:,:,:]& L_p, bint[:,:]& L_h, double[:,:]& L_c,
                     bint[:,:,:]& L_reach, bint[:,:]& L_labeled, int[:,:]& L_idx):
    cdef size_t i ,j
    #update the path
    for i in range(L_idx[prnt_i][prnt_v]+1):
        L_p[chd_i][i][chd_v] = L_p[prnt_i][i][prnt_v]
    L_p[chd_i][L_idx[prnt_i][prnt_v]+1][chd_v] = chd_v

    #update index of the last element of the path
    L_idx[chd_i][chd_v] = L_idx[prnt_i][prnt_v]+1

    #update the resources and check the half-point
    for j in range(num_resources):
        if drct ==1:
            L_r[chd_i][j][chd_v] = r[prnt_v][chd_v][j]+L_r[prnt_i][j][prnt_v]
            if L_r[chd_i][j][chd_v]>=r_max[j]/2:
                L_h[chd_i][chd_v] = 1
        else:
            L_r[chd_i][j][chd_v] = -r[prnt_v][chd_v][j]+L_r[prnt_i][j][prnt_v]
            if L_r[chd_i][j][chd_v]<=r_max[j]/2:
                L_h[chd_i][chd_v] = 1
    if drct==1:
        L_c[chd_i][chd_v] = L_c[prnt_i][prnt_v]+c[prnt_v][chd_v]
    else:
        L_c[chd_i][chd_v] = L_c[prnt_i][prnt_v]+c[chd_v][prnt_v]
    #update the reachables
    L_reach[chd_i][chd_v][chd_v] = 0
    
    #change the label from new into old
    L_labeled[chd_i][chd_v] = 1

    #update the reachables
    for i in range(num_nodes):
        if i!=chd_v:
            if L_reach[prnt_i][i][prnt_v]==0:
                L_reach[chd_i][i][chd_v] = L_reach[prnt_i][i][prnt_v]
            else:
                L_reach[chd_i][i][chd_v] = 1
                for j in range(num_resources):
                    if drct==1:
                        if r[chd_v][i][j]+L_r[chd_i][j][chd_v]>r_max[j]:
                            L_reach[chd_i][i][chd_v] = 0
                            break
                    else:
                        if L_r[chd_i][j][chd_v]-r[i][chd_v][j]<0:
                            L_reach[chd_i][i][chd_v] = 0
                            break
    
cdef void WhichP2Propogate(size_t prnt_v, double[:, :] &L_c, bint[:, :] &L_labeled,
                           bint[:, :] &L_childup, bint[:, :] &L_h, size_t[:] &idx_max, size_t *min_index):
    cdef size_t i
    cdef double min_c = float('inf')  # Use infinity for initialization
    *min_index = -1  # Properly set the dereferenced pointer value
    
    for i in range(idx_max[prnt_v] + 1):
        if (L_labeled[i, prnt_v] == 0 and
            L_childup[i, prnt_v] == 0 and
            L_h[i, prnt_v] == 0 and
            L_c[i, prnt_v] < min_c):
            *min_index = i
            min_c = L_c[i, prnt_v]

cdef void Continuoue2Search(int num_nodes, double[:, :] &L_c, bint[:, :] &L_labeled,
                            bint[:, :] &L_childup, bint[:, :] &L_h, size_t[:] &idx_max, size_t *prnt_index, size_t *vertex):
    cdef size_t v, indx
    *prnt_index = -1  # Initialize pointers
    *vertex = -1

    for v in range(num_nodes):
        indx = -1
        WhichP2Propogate(v, L_c, L_labeled, L_childup, L_h, idx_max, &indx)

        if indx != -1:
            *vertex = v
            *prnt_index = indx
            break















    




    






    # mom2calc[:] = [1, 2, 3]


Error compiling Cython file:
------------------------------------------------------------
...
    
cdef void WhichP2Propogate(size_t prnt_v, double[:, :] &L_c, bint[:, :] &L_labeled,
                           bint[:, :] &L_childup, bint[:, :] &L_h, size_t[:] &idx_max, size_t *min_index):
    cdef size_t i
    cdef double min_c = float('inf')  # Use infinity for initialization
    *min_index = -1  # Properly set the dereferenced pointer value
    ^
------------------------------------------------------------

C:\Users\rezam\.ipython\cython\_cython_magic_a218f3e5fc7d5bf1ac958914ac0df780fb5246f3.pyx:111:4: a starred assignment target must be in a list or tuple - maybe you meant to use an index assignment: var[0] = ...
